# Example 1

### End-to-end workflow to calculate conformer proportions and NMR spectra of strychnine with AQME from a SMILES string

This workflow includes:

i) RDKit conformer sampling \
ii) Gaussian geometry optimization (B3LYP/6-31+G(d,p)) \
iii) Fixing errors and imaginary frequencies of the output files \
iv) Calculation and analysis of NMR chemical shifts for the conformers generated \
v) GoodVibes calculation of Boltzmann distributions using Gibbs free energies at 298.15 K

#### Steps involved in this example

- Step 1: Importing AQME and other python modules
- Step 2: CSEARCH conformational sampling
- Step 3: Creating Gaussian input files for optimization and frequency with QPREP
- Step 4: Running Gaussian inputs for optimization and frequency calcs externally
- Step 5: QCORR analysis including isomerization filter
- Step 6: Resubmission of unsuccessful calculations (if any) with suggestions from AQME
- Step 7: Creating Gaussian input files for NMR calcs with QPREP
- Step 8: Running Gaussian NMR calcs
- Step 9: Obtaining Boltzmann weighted NMR shifts with QDESCP
- Step 10: Calculating conformer populations with GoodVibes

###  Step 1: Importing AQME and other python modules

In [2]:
import os, subprocess,shutil, glob
from pathlib import Path 
from aqme.csearch import csearch
from aqme.qprep import qprep
from aqme.qcorr import qcorr
from aqme.qdescp import qdescp

### Step 2: CSEARCH conformational sampling

In [ ]:
name = 'Strychnine'
smi = 'C1CN2CC3=CCO[C@H]4CC(=O)N5[C@H]6[C@H]4[C@H]3C[C@H]2[C@@]61C7=CC=CC=C75'
program = 'rdkit'

sdf_path = f'{os.getcwd()}/{name}_sdf_files' # folder where the SDF files are generated

csearch(destination=sdf_path,program=program,smi=smi,name=name)

### Step 3: Creating Gaussian input files for optimization and frequency with QPREP

In [ ]:
program = 'gaussian'
qm_input = 'B3LYP/6-31+G(d,p) opt freq'
mem='24GB'
nprocs=12

sdf_rdkit_files = f'{sdf_path}/*.sdf' # SDF files from Step 2
com_path = f'{os.getcwd()}/{name}_com_files' # folder where the COM files are generated

qprep(destination=com_path,files=sdf_rdkit_files,program=program,
        qm_input=qm_input,mem=mem,nprocs=nprocs)

### Step 4: Running Gaussian inputs for optimization and frequency calcs externally

In [ ]:
# Run the generated COM files (in com_path, see PATH in Step 3) with Gaussian

### Step 5: QCORR analysis including isomerization filter

In [ ]:
log_files=f'{com_path}/*.log' # LOG files from Step 4

qcorr(files=log_files,freq_conv='opt=(calcfc,maxstep=5)',isom_type='com',isom_inputs=com_path,nprocs=12,mem='24GB')

### Step 6: Resubmission of unsuccessful calculations (if any) with suggestions from AQME

In [ ]:
# Run the generated COM files (in fixed_QM_inputs, shown as resub_path in Step 7) with Gaussian

### Step 7: QCORR analysis of unsuccessful calculations (if any) from Step 6

In [ ]:
resub_path = f'{com_path}/failed/run_1/fixed_QM_inputs' # LOG files from Step 6
log_files_resub = f'{resub_path}/*.log'

# this QCORR analysis skips the freq_conv since we noticed that thermochemistry data barely change after one freq_conv correction
# if there are no files that failed, you will see a PATH error
qcorr(files=log_files_resub,isom_type='com',isom_inputs=com_path,nprocs=12,mem='24GB')

### Step 8: Creating Gaussian input files for NMR calcs with QPREP

In [ ]:
program = 'gaussian'
qm_input = 'B3LYP/6-311+G(2d,p) scrf=(solvent=chloroform,smd) nmr=giao'
mem='24GB'
nprocs=12

success_folder = com_path+'/success' # folder where the successful LOG files are stored during the QCORR cycles (Steps 5 and 6)
log_files = f'{success_folder}/*.log'
sp_path = f'{os.getcwd()}/{name}_sp_files' # folder to store the new COM inputs for single point NMR calcs

qprep(w_dir_main=success_folder,destination=sp_path,files=log_files,program=program,qm_input=qm_input,mem=mem,nprocs=nprocs,suffix='SP',nprocs=12,mem='24GB')

### Step 9: Running Gaussian NMR calcs

In [ ]:
# Run the generated COM files (in sp_path, see PATH in Step 7) with Gaussian

### Step 10: Obtaining Boltzmann weighted NMR shifts with QDESCP

In [ ]:
# Create JSON files with QCORR to store the information from the resulting LOG files
log_files=f'{sp_path}/*.log'
qcorr(files=log_files)

# Analyze the JSON files to calculate the Boltzmann averaged shielding tensors
json_folder = sp_path+'/success/SP_calcs/json_files' # folder where the JSON files were just created with QCORR
json_files=f'{json_folder}/*.json'
nmr_path = f'{os.getcwd()}/{name}_nmr_files' # folder to store the results from QDESCP

qdescp(program='nmr',boltz=True,files=json_files,destination=nmr_path,nmr_slope=[-1.0537, -1.0784],nmr_intercept=[181.7815,31.8723], nmr_experim='Experimental_NMR_shifts.csv')

### Step 11: Calculating conformer populations with GoodVibes

In [ ]:
log_files = glob.glob(f'{success_folder}/*.log')
log_files += glob.glob(f'{sp_path}/success/SP_calcs/*.log')

w_dir_main  = Path(os.getcwd())
GV_folder = w_dir_main.joinpath('Strychine_GoodVibes-analysis')
GV_folder.mkdir(exist_ok=True, parents=True)

for file in log_files:
	shutil.copy(file, GV_folder)

# run GoodVibes
os.chdir(GV_folder)
subprocess.run(['python', '-m', 'goodvibes', '--xyz', '-c', '1', '*.log','--boltz', '--spc', 'SP'])
os.chdir(w_dir_main)
